<a href="https://colab.research.google.com/github/Gvascons/Parking_Spot_Detection/blob/main/Time_Series_Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import re

In [ ]:
dr_path = '/content/drive/MyDrive/ParkingLotDL/'

### CNRPark

In [ ]:
mdCNR = pd.read_csv(dr_path + 'CNRPark/CNRPark+EXT.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
mdCNR.columns

Index(['camera', 'datetime', 'day', 'hour', 'image_url', 'minute', 'month',
       'occupancy', 'slot_id', 'weather', 'year', 'occupant_changed'],
      dtype='object')

In [ ]:
mdCNR.drop_duplicates(subset=['slot_id', 'datetime'], inplace=True)

,camera,datetime,day,hour,image_url,minute,month,occupancy,slot_id,weather,year,occupant_changed


In [ ]:
occupancyCNRPark = mdCNR.groupby(['datetime']).agg({'occupancy': 'sum', 'weather': 'first'})

In [ ]:
occupancyCNRPark.index = [t[:4]+'-'+t[4:6]+'-'+t[6:11]+'.'+t[-2:] if len(t) == 13 else t for t in list(occupancyCNRPark.index)]

In [ ]:
occupancyCNRPark.sort_index(inplace=True)

In [ ]:
occupancyCNRPark

,occupancy,weather
2015-07-03_08.05,6,S
2015-07-03_08.10,7,S
2015-07-03_08.15,8,S
2015-07-03_08.20,8,S
2015-07-03_08.25,9,S
...,...,...
2016-02-12_17.35,4,R
2016-02-12_17.36,10,R
2016-02-12_17.41,20,R
2016-02-12_17.42,14,R


In [ ]:
occupancyCNRPark.to_csv(dr_path + 'CNRPark/occupancyCNRPark.csv')

### PKLot

In [ ]:
dictDay = {
    'datetime': [],
    'occupied': [],
    'parkinglot': [],
    'weather': []
}
for pks in os.listdir(dr_path + 'PKLot'):
  print(pks)
  for clim in os.listdir(dr_path + 'PKLot/' + pks):
    print(clim)
    for day in os.listdir(dr_path + 'PKLot/' + pks + '/' + clim):
      print(day)
      dayFiles = [f for f in os.listdir(dr_path + 'PKLot/' + pks + '/' + clim + '/' + day) if f.endswith('.xml')]
      for file in dayFiles:
        f = open(dr_path + 'PKLot/' + pks + '/' + clim + '/' + day + '/' + file)
        xml = f.read()
        occ = re.findall(r"(?<=occupied=\")1", xml)
        dictDay['datetime'].append(file[:-4])
        dictDay['occupied'].append(sum(list(map(int, occ))))
        dictDay['parkinglot'].append(pks)
        dictDay['weather'].append(clim)

FileNotFoundError: ignored

In [ ]:
len(dictDay['occupied'])

12416

In [ ]:
occupancyPKLot = pd.DataFrame.from_dict(dictDay)

In [ ]:
occupancyPKLot.sort_values(by='datetime', inplace=True)

In [ ]:
occupancyPKLot

,datetime,occupied,parkinglot,weather
9098,2012-09-11_15_16_58,67,PUCPR,Sunny
9113,2012-09-11_15_27_08,67,PUCPR,Sunny
9114,2012-09-11_15_29_29,69,PUCPR,Sunny
9100,2012-09-11_15_31_50,69,PUCPR,Sunny
9091,2012-09-11_15_36_32,69,PUCPR,Sunny
...,...,...,...,...
6215,2013-04-16_10_30_05,38,UFPR05,Sunny
6184,2013-04-16_10_35_05,39,UFPR05,Sunny
6172,2013-04-16_10_40_05,39,UFPR05,Sunny
6198,2013-04-16_10_45_05,38,UFPR05,Sunny


In [ ]:
occupancyPKLot.to_csv(dr_path + 'PKLot/occupancyPKLot.csv')